In [29]:
print("bobnet debug")

bobnet debug


In [30]:
from fetch_it import mnist

x_train,y_train, x_test,y_test = mnist()

In [31]:
import numpy as np

In [32]:
from frame import layer_init

def forward_backward(x,y,l1,l2):
    f_l1 = x@l1 # (1,128)
    f_l2 = f_l1@l2 # (1,10)
    # backward
    # loss calculation: binary cross entropy w/ softmax
    out = np.zeros((len(y),10),np.float32)
    out[range(out.shape[0]),y] = 1

    pp = (f_l2 - np.log(np.exp(f_l2)).sum(axis=1).reshape((-1,1)))
    assert out.shape == pp.shape
    loss = -np.multiply(out, pp).mean(axis=1)
    #assert loss > 0
    # backprop
    d_out = -out/len(y)
    grads = d_out - (np.exp(-loss)*d_out.sum(axis=1)).reshape((-1,1))
    d_l2 = f_l1.T@grads
    assert d_l2.shape == l2.shape
    d_l1 = x.T@((l2@grads.T).T)
    assert d_l1.shape == l1.shape
    return f_l2, loss, d_l1, d_l2

In [33]:
from tqdm import tqdm

np.random.seed(1337)

l1 = layer_init(784,128)
l2 = layer_init(128,10)

losses = []
lr = 1e-6
for i in tqdm(range(1000)):
    samp = np.random.randint(0,x_train.shape[0],size=(32))
    X = x_train[samp].reshape((-1,28**2))
    Y = y_train[samp]
    pred, loss, d_l1, d_l2 = forward_backward(X,Y,l1,l2)
    l1 -= lr*d_l1
    l2 -= lr*d_l2

    losses.append(loss.mean())

losses[:10], pred[0]

  0%|          | 0/1000 [00:00<?, ?it/s]<ipython-input-32-570640b0185b>:11: RuntimeWarning: divide by zero encountered in log
  pp = (f_l2 - np.log(np.exp(f_l2)).sum(axis=1).reshape((-1,1)))
<ipython-input-32-570640b0185b>:13: RuntimeWarning: invalid value encountered in multiply
  loss = -np.multiply(out, pp).mean(axis=1)
100%|██████████| 1000/1000 [00:00<00:00, 2724.90it/s]


([0.00593465,
  -0.036825538,
  -0.10968059,
  -0.14623457,
  -0.20979637,
  -0.36515918,
  -0.53245527,
  -0.58392674,
  -0.65903664,
  -0.8064263],
 array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan], dtype=float32))